# 1. Import Data

## Import dependencies, set up environment

1. `python3 -m venv .venv`
2. utilize virtual environment
    - (LINUX/MAC) `source .venv/bin/activate`
    - (WINDOWS) `.venv\Scripts\Activate.    ps1`
3. `cd project_folder/util`
4. `pip install -r requirements.txt`

In [3]:
# Jupyter magic
%run ../util/dependencies.py

## FAA Service Discrepancy Reports

In [ ]:
filepath = '../data/01_raw'
filenames = os.listdir(filepath)
filenames = [filepath + '/' + f for f in filenames]

# try opening first file
if(not pd.read_html(filenames[0])):
   print('ERROR LOADING FILES')

df = pd.read_html(filenames[0])[0]

# concat other files
if len(filenames) > 1:
    for f in filenames[1:]:
        df = pd.concat([df, pd.read_html(f)[0]], ignore_index=True)

['../data/01_raw/202503.xls', '../data/01_raw/202501.xls', '../data/01_raw/202502.xls']
['../data/01_raw/202501.xls', '../data/01_raw/202502.xls']
<class 'pandas.DataFrame'>
RangeIndex: 3258 entries, 0 to 3257
Data columns (total 76 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   OperatorControlNumber    3258 non-null   str    
 1   DifficultyDate           3258 non-null   str    
 2   SubmissionDate           3258 non-null   str    
 3   OperatorDesignator       3247 non-null   str    
 4   SubmitterDesignator      3247 non-null   str    
 5   SubmitterTypeCode        3258 non-null   str    
 6   ReceivingRegionCode      3258 non-null   str    
 7   ReceivingDistrictOffice  3258 non-null   int64  
 8   SDRType                  3258 non-null   str    
 9   JASCCode                 3258 non-null   int64  
 10  NatureOfConditionA       3258 non-null   str    
 11  NatureOfConditionB       70 non-null     str    
 

In [ ]:
df.to_csv("../data/02_csv/SDR_COMPOSITE_EXPORT.csv", index=False)

OperatorControlNumber       0
DifficultyDate              0
SubmissionDate              0
OperatorDesignator         11
SubmitterDesignator        11
                         ... 
CrackLength              3150
NumberOfCracks           3087
CorrosionLevel           2714
StructuralOther          3219
Discrepancy                 0
Length: 76, dtype: int64


## Supply Chain Tables

In [4]:
# -----------------------------
# 1. Load SDR composite CSV
# -----------------------------
sdr_path = "../data/02_csv/SDR_COMPOSITE_EXPORT.csv"
sdr = pd.read_csv(sdr_path)

# Normalize column names just in case
sdr.columns = [c.strip() for c in sdr.columns]

# Key fields we’ll use if present
# From your sample: JASCCode, PartMake, PartName, PartNumber, PartCondition,
# ComponentMake, ComponentName, ComponentPartNumber, ComponentLocation, Discrepancy

# -----------------------------
# 2. Build unified "parts universe"
# -----------------------------
# A) From Part* fields
parts_a = sdr[[
    "JASCCode",
    "PartMake",
    "PartName",
    "PartNumber",
    "PartCondition",
    "PartLocation"
]].copy()

parts_a.rename(columns={
    "PartMake": "part_make",
    "PartName": "part_name",
    "PartNumber": "part_number",
    "PartCondition": "condition",
    "PartLocation": "location",
    "JASCCode": "jasc_code"
}, inplace=True)

# B) From Component* fields
parts_b = sdr[[
    "JASCCode",
    "ComponentMake",
    "ComponentName",
    "ComponentPartNumber",
    "ComponentLocation"
]].copy()

parts_b.rename(columns={
    "ComponentMake": "part_make",
    "ComponentName": "part_name",
    "ComponentPartNumber": "part_number",
    "ComponentLocation": "location",
    "JASCCode": "jasc_code"
}, inplace=True)

parts_b["condition"] = np.nan  # often not explicitly given

# C) Stack them
parts_all = pd.concat([parts_a, parts_b], ignore_index=True)

# Drop rows with no name at all
parts_all = parts_all.dropna(subset=["part_name"])

# Fill missing part numbers with a synthetic ID so we can still track them
parts_all["part_number"] = parts_all["part_number"].fillna("UNKNOWN")

# Deduplicate by (part_number, part_name, jasc_code)
parts_unique = (
    parts_all
    .drop_duplicates(subset=["part_number", "part_name", "jasc_code"])
    .reset_index(drop=True)
)

# -----------------------------
# 3. Create synthetic suppliers
# -----------------------------
num_suppliers = 8
suppliers = pd.DataFrame({
    "supplier_id": [f"S{i}" for i in range(1, num_suppliers + 1)],
    "supplier_name": [f"Supplier_{i}" for i in range(1, num_suppliers + 1)],
    "on_time_delivery_rate": np.round(np.random.uniform(0.85, 0.99, num_suppliers), 3),
    "avg_delay_days": np.random.randint(1, 21, num_suppliers),
    "quality_rating": np.round(np.random.uniform(3.0, 5.0, num_suppliers), 2),
})

# Assign suppliers based (loosely) on JASC band so it's not purely random
def supplier_for_jasc(jasc):
    try:
        j = int(str(jasc)[:2])
    except (ValueError, TypeError):
        return np.random.choice(suppliers["supplier_id"])
    # Group JASC bands to supplier ranges
    if j in [21, 22, 23, 24]:
        return np.random.choice(suppliers["supplier_id"][:3])
    elif j in [25, 26, 27, 28, 29]:
        return np.random.choice(suppliers["supplier_id"][2:6])
    else:
        return np.random.choice(suppliers["supplier_id"])

parts_unique["supplier_id"] = parts_unique["jasc_code"].apply(supplier_for_jasc)

# -----------------------------
# 4. Lead times, unit costs, criticality
# -----------------------------
def lead_time_for_jasc(jasc):
    try:
        j = int(str(jasc)[:2])
    except (ValueError, TypeError):
        return np.random.randint(10, 45)
    if j in [21, 22, 23]:          # air conditioning, auto flight, comm/nav
        return np.random.randint(7, 14)
    elif j in [25, 26, 27]:        # equipment/furnishings, fire, flight controls
        return np.random.randint(20, 45)
    elif j in [32, 33, 34]:        # landing gear, lights, nav
        return np.random.randint(45, 90)
    elif j in [52, 53, 54]:        # doors, fuselage, nacelles
        return np.random.randint(30, 75)
    else:
        return np.random.randint(10, 60)

def cost_for_row(row):
    jasc = row["jasc_code"]
    name = str(row["part_name"]).upper()
    try:
        j = int(str(jasc)[:2])
    except (ValueError, TypeError):
        j = None

    # Heuristic by type
    if "SLIDE" in name:
        return np.random.randint(15000, 45000)
    if "FLOORBEAM" in name or "FLOOR BEAM" in name:
        return np.random.randint(5000, 25000)
    if "BATTERY" in name:
        return np.random.randint(500, 5000)
    if "POWER SUPPLY" in name:
        return np.random.randint(2000, 10000)
    if "LAMP" in name or "LIGHT" in name:
        return np.random.randint(100, 1500)

    # Fallback by JASC band
    if j in [21, 22, 23]:
        return np.random.randint(500, 8000)
    elif j in [25, 26, 27]:
        return np.random.randint(2000, 20000)
    elif j in [32, 33, 34]:
        return np.random.randint(10000, 100000)
    elif j in [52, 53, 54]:
        return np.random.randint(5000, 50000)
    else:
        return np.random.randint(1000, 15000)

def criticality_for_row(row):
    jasc = row["jasc_code"]
    cond = str(row.get("condition", "")).upper()
    name = str(row["part_name"]).upper()
    try:
        j = int(str(jasc)[:2])
    except (ValueError, TypeError):
        j = None

    # Condition-based bump
    if any(k in cond for k in ["CRACK", "INOPERATIVE", "FAULT", "FAILED"]):
        base = "High"
    elif "CORROD" in cond:
        base = "Medium"
    else:
        base = "Low"

    # ATA-based override for safety-critical systems
    if j in [27, 32, 33, 34]:
        return "AOG-critical"
    if j in [25, 26, 52, 53]:
        return "High" if base != "Low" else "Medium"

    return base

parts_unique["lead_time_days"] = parts_unique["jasc_code"].apply(lead_time_for_jasc)
parts_unique["unit_cost"] = parts_unique.apply(cost_for_row, axis=1)
parts_unique["criticality_level"] = parts_unique.apply(criticality_for_row, axis=1)

parts_master = parts_unique[
    [
        "part_number",
        "part_name",
        "part_make",
        "jasc_code",
        "unit_cost",
        "lead_time_days",
        "criticality_level",
        "supplier_id",
        "location",
        "condition",
    ]
].copy()

# -----------------------------
# 5. Inventory table (failure-driven)
# -----------------------------
# Count failures by part_number + part_name across both Part* and Component* usage
# First, normalize SDR into a "usage" view
usage_a = sdr[["PartNumber", "PartName"]].copy()
usage_a.rename(columns={"PartNumber": "part_number", "PartName": "part_name"}, inplace=True)

usage_b = sdr[["ComponentPartNumber", "ComponentName"]].copy()
usage_b.rename(columns={"ComponentPartNumber": "part_number", "ComponentName": "part_name"}, inplace=True)

usage_all = pd.concat([usage_a, usage_b], ignore_index=True)
usage_all = usage_all.dropna(subset=["part_name"])
usage_all["part_number"] = usage_all["part_number"].fillna("UNKNOWN")

failure_counts = (
    usage_all
    .groupby(["part_number", "part_name"])
    .size()
    .reset_index(name="failure_count")
)

inventory = parts_master.merge(
    failure_counts,
    on=["part_number", "part_name"],
    how="left"
)

inventory["failure_count"] = inventory["failure_count"].fillna(0)

def stock_from_row(row):
    f = row["failure_count"]
    crit = row["criticality_level"]
    cost = row["unit_cost"]
    lt = row["lead_time_days"]

    # Base on failures
    if f == 0:
        base = 0
    elif f < 3:
        base = 1
    elif f < 10:
        base = 3
    else:
        base = 5

    # Adjust for criticality and lead time
    if crit == "AOG-critical":
        base = max(base, 3)
    if lt > 45:
        base = max(base, 3)

    # Adjust down for very high cost
    if cost > 50000 and base > 3:
        base = 3

    return base

inventory["stock_on_hand"] = inventory.apply(stock_from_row, axis=1)
inventory["reorder_point"] = (inventory["stock_on_hand"] // 2).clip(lower=0)
inventory["reorder_quantity"] = (inventory["stock_on_hand"] * 2).clip(lower=0)
inventory["backorder_flag"] = inventory["stock_on_hand"] == 0

inventory = inventory[
    [
        "part_number",
        "part_name",
        "stock_on_hand",
        "reorder_point",
        "reorder_quantity",
        "backorder_flag",
        "failure_count",
        "criticality_level",
        "unit_cost",
        "lead_time_days",
        "supplier_id",
    ]
].copy()

# -----------------------------
# 6. Save outputs
# -----------------------------
parts_master.to_csv("../data/02_csv/parts_master.csv", index=False)
suppliers.to_csv("../data/02_csv/suppliers.csv", index=False)
inventory.to_csv("../data/02_csv/inventory.csv", index=False)

print("Generated: parts_master.csv, suppliers.csv, inventory.csv")


Generated: parts_master.csv, suppliers.csv, inventory.csv


In [6]:
# -----------------------------
# 1. Load existing tables
# -----------------------------
sdr = pd.read_csv("../data/02_csv/SDR_COMPOSITE_EXPORT.csv")
parts_master = pd.read_csv("../data/02_csv/parts_master.csv")
inventory = pd.read_csv("../data/02_csv/inventory.csv")

# -----------------------------
# 2. Normalize SDR fields
# -----------------------------
sdr["PartNumber"] = sdr["PartNumber"].fillna("UNKNOWN").astype(str).str.upper().str.strip()
sdr["PartName"] = sdr["PartName"].fillna("UNKNOWN").astype(str).str.upper().str.strip()

# Normalize parts_master
parts_master["part_number"] = parts_master["part_number"].astype(str).str.upper().str.strip()
parts_master["part_name"] = parts_master["part_name"].astype(str).str.upper().str.strip()

# Normalize inventory
inventory["part_number"] = inventory["part_number"].astype(str).str.upper().str.strip()
inventory["part_name"] = inventory["part_name"].astype(str).str.upper().str.strip()

# -----------------------------
# 3. Create SDR join key
# -----------------------------
sdr_key = sdr[["OperatorControlNumber", "JASCCode", "PartNumber", "PartName"]].copy()
sdr_key.rename(columns={
    "OperatorControlNumber": "sdr_id",
    "JASCCode": "jasc_code",
    "PartNumber": "part_number",
    "PartName": "part_name"
}, inplace=True)

# -----------------------------
# 4. Primary join: part_number + part_name
# -----------------------------
merged = sdr_key.merge(
    parts_master,
    on=["part_number", "part_name"],
    how="left",
    suffixes=("", "_pm")
)

# -----------------------------
# 5. Secondary join: part_name + jasc_code
#    (fills missing metadata)
# -----------------------------
missing = merged["criticality_level"].isna()

fallback = sdr_key[missing].merge(
    parts_master,
    on=["part_name", "jasc_code"],
    how="left"
)

# Fill missing fields from fallback
for col in ["unit_cost", "lead_time_days", "criticality_level", "supplier_id", "location", "condition"]:
    merged.loc[missing, col] = fallback[col].values

# -----------------------------
# 6. Join inventory (stocking data)
# -----------------------------
merged = merged.merge(
    inventory,
    on=["part_number", "part_name"],
    how="left",
    suffixes=("", "_inv")
)

# -----------------------------
# 7. Fill remaining missing values with defaults
# -----------------------------
merged["criticality_level"] = merged["criticality_level"].fillna("Medium")
merged["unit_cost"] = merged["unit_cost"].fillna(5000)
merged["lead_time_days"] = merged["lead_time_days"].fillna(30)
merged["supplier_id"] = merged["supplier_id"].fillna("S0")
merged["stock_on_hand"] = merged["stock_on_hand"].fillna(0)
merged["failure_count"] = merged["failure_count"].fillna(0)

# -----------------------------
# 8. Synthetic Work-Order Logic
# -----------------------------
def maintenance_type_for_jasc(jasc):
    try:
        j = int(str(jasc)[:2])
    except:
        return "Line"
    if j in [25, 33, 52]:
        return "Line"
    if j in [53, 54]:
        return "Base"
    return np.random.choice(["Line", "Overnight"])

def repair_duration(row):
    cond = str(row.get("condition", "")).upper()
    jasc = row["jasc_code"]

    base = 2  # hours

    if "CRACK" in cond:
        base = 12
    elif "CORROD" in cond:
        base = 8
    elif "INOPER" in cond or "FAULT" in cond:
        base = 4
    elif "LEAK" in cond:
        base = 3

    try:
        j = int(str(jasc)[:2])
        if j in [53, 54]:  # structural
            base += 10
    except:
        pass

    return base

def aog_flag(row):
    if row["criticality_level"] == "AOG-critical":
        return np.random.choice([1, 0], p=[0.7, 0.3])
    return np.random.choice([0, 1], p=[0.8, 0.2])

def backorder_delay(row):
    if row["stock_on_hand"] > 0:
        return 0
    lt = row["lead_time_days"]
    return max(1, int(np.random.normal(lt * 0.4, 2)))

def labor_hours(row):
    dur = row["repair_duration_hours"]
    return round(dur * np.random.uniform(0.8, 1.4), 1)

def deferred_flag(row):
    if row["criticality_level"] == "Low":
        return np.random.choice([1, 0], p=[0.4, 0.6])
    return 0

# Apply synthetic logic
merged["maintenance_type"] = merged["jasc_code"].apply(maintenance_type_for_jasc)
merged["repair_duration_hours"] = merged.apply(repair_duration, axis=1)
merged["aog_flag"] = merged.apply(aog_flag, axis=1)
merged["backorder_delay_days"] = merged.apply(backorder_delay, axis=1)
merged["labor_hours"] = merged.apply(labor_hours, axis=1)
merged["turnaround_time_hours"] = merged["repair_duration_hours"] + (merged["backorder_delay_days"] * 24)
merged["deferred_flag"] = merged.apply(deferred_flag, axis=1)

def priority(row):
    if row["aog_flag"] == 1:
        return "High"
    if row["criticality_level"] in ["High", "AOG-critical"]:
        return "High"
    if row["criticality_level"] == "Medium":
        return "Medium"
    return "Low"

merged["priority"] = merged.apply(priority, axis=1)

# -----------------------------
# 9. Add work-order ID
# -----------------------------
merged["wo_id"] = ["WO" + str(i).zfill(6) for i in range(len(merged))]

# -----------------------------
# 10. Final selection
# -----------------------------
work_orders = merged[[
    "wo_id",
    "sdr_id",
    "part_number",
    "part_name",
    "jasc_code",
    "maintenance_type",
    "repair_duration_hours",
    "aog_flag",
    "backorder_delay_days",
    "labor_hours",
    "turnaround_time_hours",
    "deferred_flag",
    "priority"
]]

# -----------------------------
# 11. Save
# -----------------------------
work_orders.to_csv("../data/02_csv/work_orders.csv", index=False)

print("Generated: work_orders.csv")


/tmp/ipykernel_6287/2036754632.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fallback = sdr_key[missing].merge(


Generated: work_orders.csv
